In [ ]:
import pandas as pd
import seaborn as sns
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
sms = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding='latin-1')
sms.head()

#Dataframe possède 5 colonnes

In [ ]:
#Surppression des 3 colonnes "Unnamed"

sms.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)
sms.head()

In [ ]:
#Renommer la colonne "v1" par "label" et la colonne "v2" par "message" et affiche du résultat

sms=sms.rename(columns={"v1":"label","v2":"message"})

In [ ]:
#Dataframe avec les colonnes renommées

sms.head()

In [ ]:
#Diagramme de la colonne label avec la fonction countplot
#Affichage de ka quatité de Ham et Spam

sns.countplot(sms.label)

In [ ]:
#Afficher la taille de la DataFrame
sms.shape

In [ ]:
#Réduction de la taille de la police en minuscules
sms['message']=sms['message'].str.lower()

In [ ]:
#Supprimession des ponctuations

def remove_punct(text):
    text_tok = word_tokenize(text)
    l=[]
    for word in text_tok: 
        if not word in string.punctuation:
            l.append(word)
           
    resultat=" ".join(l)  
    return resultat

sms['message']=sms.message.apply(remove_punct)

In [ ]:
#Supprimession des Stop words 

stop=set(stopwords.words('english'))

def remove_stopword(text):
    #On divise le texte en morceau
    text_tok = word_tokenize(text)
    #Initialisation liste vide
    l = []
    for a in text_tok:
        if not a in stop:
            l.append(a)
            
    resultat = " ".join(l)
    return resultat

sms['message']=sms.message.apply(remove_stopword)

In [ ]:
#Remplacer les mots par leur forme canonique 

lemmatizer=WordNetLemmatizer()

def lemm(text):
    text_tok = word_tokenize(text) 
    l=[]
    for word in text_tok:
        l.append(lemmatizer.lemmatize(word))
        
    resultat = " ".join(l)

    return resultat

sms.message=sms.message.apply(lemm)

## Bag of words

In [ ]:
# Vectoriser la colonne des messages par la méthode Bag of words

In [ ]:
x#Impirtation de la fonction CoutVectorizer de la bibliothèque sklearn.feature_extraction.text
from sklearn.feature_extraction.text import CountVectorizer

#Corpus prend la valeur de la colonne message du tableau sms

corpus=sms['message'].values
bw_vect = CountVectorizer()
# tokenize et construire le vocabulaire
bw_fit=bw_vect.fit(corpus)
# vectoriser les mots
bw_corpus = bw_fit.transform(corpus)
bw_sms=pd.DataFrame(bw_corpus.toarray(),columns=bw_fit.get_feature_names())
bw_sms

## TF IDF

In [ ]:
#Vectoriser la colonne des messages par le méthode TF IDF

from sklearn.feature_extraction.text import TfidfVectorizer
#Initialiser les paramètres du vectoriseur
tf_vect = TfidfVectorizer(max_features=500)
#Apprendre le vocabulaire du vectoriseur basé sur le paramètre initialisé
tfidf_fit=tf_vect.fit(corpus)
#Vectoriser le corpus
tfidf_corpus= tfidf_fit.transform(corpus)
tfidf_sms=pd.DataFrame(tfidf_corpus.toarray(),columns=tfidf_fit.get_feature_names())
tfidf_sms

## 1ère méthode : avec TFIDF

## Vectorisation

In [ ]:
from sklearn.model_selection import train_test_split
Xtfidf=tfidf_sms
Y=sms.label
# Split train / test data :
X_traintfidf, X_testtfidf, Y_train, Y_test = train_test_split(Xtfidf, Y, test_size=0.3, random_state=0)

## Arbre de décision

In [ ]:
#Importation de la fonction tree de la bibliothèque sklearn

from sklearn import tree
tree_model = tree.DecisionTreeClassifier()
tree_model = tree.DecisionTreeClassifier(max_depth = 2)
tree_model = tree_model.fit(X_traintfidf, Y_train)

In [ ]:
#Importation des librairies matplotlib.pyplot et renommer en plt

import matplotlib.pyplot as plt
plt.figure(figsize=(15,10))
names = ['spam', 'non spam']
tree.plot_tree(tree_model,feature_names = Xtfidf.columns, 
               class_names=names,
               filled = True)

# Affichage de l'arbre de trie entre les spam et ham

In [ ]:
#Création de la variable de prédilection

Y_predicttfidf=tree_model.predict(X_testtfidf)

## Évaluation de l'arbre

In [ ]:
# En utilisant l'arbre de décision il faut deviner si chaque message de la variable Y_test est spam ou non
# Il faut sauvegarder la réponse dans la variable Y_prredicttfidf

from sklearn.metrics import accuracy_score, confusion_matrix 
mat = confusion_matrix(Y_predicttfidf, Y_test)
print(mat)

#Afficher mat

In [ ]:
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
plt.xlabel('Test')
plt.ylabel('Predicted')

#Création du tableau permettant de comparer les valeurs prédites avec les valeurs déjà entrées

## 2ème méthode : avec Bag of words

## Vectorisation

In [ ]:
#Importation de la fonction train_test_split de la bibliothèque sklearn.model_selection

from sklearn.model_selection import train_test_split
Xbw=bw_sms
Y=sms.label
# Split train / test data : on récupère les sms du tableau et on explique au logiciel ce qu'est un spam 
X_trainbw, X_testbw, Y_train, Y_test = train_test_split(Xbw, Y, test_size=0.3, random_state=0)

## Arbre de décision

Arbre de décision est un arbre orienté dont les noeuds sont étiquetés par un test et les ars contiennent mes résultats du test. On choisit de faire un test sur la variable qui disperse le mieux les classes. Pour cela, on calcule le coefficient de Gini qui mesure l'impurité d'un sous ensemble de donnée.

In [ ]:
#Importation de la fonction tree de la bibliothèque sklearn

from sklearn import tree
tree_model = tree.DecisionTreeClassifier()

#Choisir le nombre d'étapes de l'arbre, sa profondeur 

tree_model = tree.DecisionTreeClassifier(max_depth = 2)
tree_model = tree_model.fit(X_trainbw, Y_train)

In [ ]:
#Importation de la fonction matplotlib.pyplot qui prends le nom de plt

import matplotlib.pyplot as plt
plt.figure(figsize=(15,10))
names = ['spam', 'non spam']
tree.plot_tree(tree_model,feature_names = Xbw.columns, 
               class_names=names,
               filled = True)

#Création et affichage de l'arbre de choix du logicel permettant de dire si les sms sont des spam ou ham

In [ ]:
Y_predictbw=tree_model.predict(X_testbw)

## Évaluation de l'arbre

On compare les valeurs devinées par la machine (Y_predictbw) par rapport aux vraies valeurs (Y_test)

In [ ]:
#Importation de la fonction accuracy_score et confusion_matrix de la bibliothèque sklearn.metrics
sklearn.metrics import accuracy_score, confusion_matrix

#Défintion du tableau mat qui regroupe les avelurs de Y_predict et Y_test
mat = confusion_matrix(Y_predictbw, Y_test)
print(mat)

#Afficher du tableau des valeurs de Y_predict et Y_test

In [ ]:
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
plt.xlabel('Test')
plt.ylabel('Predicted')

#Affichage du tableau comparant les valeurs prédites avec les valeurs réelles

## Suite avec la méthode TFIDF

## Gridsearch

Pour déterminer la meilleure profondeur entre 10 et 40 pour l'algorithme de l'arbre de décision

In [ ]:
#Importation de la fonction GridSearchCV de la bibliothèque sklearn.model_selection
from sklearn.model_selection import GridSearchCV
#Importation de la fonction numpy et cette dernière prend le nom de np
import numpy as np
depths = np.arange(10, 40,5)
param_grid = [{'max_depth':depths}]
grid_tree= GridSearchCV(estimator=tree.DecisionTreeClassifier(),param_grid=param_grid,scoring='accuracy',cv=10)
grid_tree.fit(X_traintfidf, Y_train)
best_model_tree = grid_tree.best_estimator_
Y_grid=best_model_tree.predict(X_testtfidf)

In [ ]:
mat = confusion_matrix(Y_grid, Y_test)
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
plt.xlabel('Test')
plt.ylabel('Predicted')

## Forêt d'arbres

Si on a un nombre important de variables explicatives (features). on utilise la Forêt d'arbres qui fonctionne comme le suivant:
- on prend des sous ensembles de données et des sous ensembles de variables explicatives.
- on applique l'Arbre de décision sur chaque sous ensemble.
- la prédiction de la forêt aléatoire est alors un simple vote majoritaire des arbes construites.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
Rf_model = RandomForestClassifier()
Rf_model=Rf_model.fit(X_traintfidf, Y_train)
Y_predicttfidf=Rf_model.predict(X_testtfidf)
a_CART = accuracy_score(Y_test,Y_predicttfidf)
print("L'accuracy score du modèle RF est de : ",a_CART)
mat = confusion_matrix(Y_predicttfidf, Y_test)
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
plt.xlabel('Test')
plt.ylabel('Predicted')

## Classer

Fonction "classer" qui prend en entrée un texte et retourne comme résultat si ce texte est spam ou non

In [ ]:
#Définition de la fonction reponse

def reponse(text):
    text=text.lower()
    text=text.replace('covid-19','coronavirus')
    text=remove_punct(text)
    text=remove_stopword(text)
    text=lemm(text)
    tfidf_text=tfidf_fit.transform([text])
   
    cm=cosine_similarity(tfidf_text, tfidf_corpus)
    pos=np.argmax(cm[0])
    data.answers[pos]
    return data.answers[pos]

## Code à tester

In [ ]:
while True:
    text = str(input("Input: "))
    if text== "exit":
        print("Response: Exiting.....")
        break
    print("Response:",reponse(text))